In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt

import cv2, numpy as np

ModuleNotFoundError: No module named 'cv2'

In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

In [ ]:
path = './'
X=[]
files = os.listdir(path)
for ix in files:
    if ix=='.DS_Store':
        continue
    X.append(np.load(os.path.join(path,ix)))
X_np = np.array(X)
print (X_np.shape)
np.save('./benign_X.npy',X_np)

In [ ]:
X_1 = np.load('./raw_benign_images_X.npy')[:499].tolist()
X_2 = np.load('./raw_malware_images_X.npy').tolist()
# print X_1.shape,X_2.shape
X= (X_1+X_2)


In [ ]:
X= np.array(X)
X = np.reshape(X,(X.shape[0],1,224,224))
y=np.zeros((X.shape[0],1))
y[:len(X_1)]=1
print (X.shape, y.shape)

In [ ]:
from keras import backend as K
K.set_image_dim_ordering('th')

In [ ]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(1,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [ ]:
model = VGG_16()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
X=X/255.0
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.2)
print (X_train.shape, y_train.shape)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=5, batch_size=4, shuffle='batch', verbose=1)

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image, ImageOps, ImageEnhance

BATCH_SIZE = 100
LEARNING_RATE = 0.01
EPOCH = 5


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding="same",stride=1),
            nn.Conv2d(1, 64, kernel_size=3, padding="same",stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding="same",stride=1),
            nn.Conv2d(64, 128, kernel_size=3, padding="same",stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding="same",stride=1),
            nn.Conv2d(128, 256, kernel_size=3, padding="same",stride=1),
            nn.Conv2d(128, 256, kernel_size=3, padding="same",stride=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(256,512, kernel_size=3, padding="same",stride=1),
            nn.Conv2d(256,512, kernel_size=3, padding="same",stride=1),
            nn.Conv2d(256,512, kernel_size=3, padding="same",stride=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(256,512, kernel_size=3, padding="same",stride=1),
            nn.Conv2d(256,512, kernel_size=3, padding="same",stride=1),
            nn.Conv2d(256,512, kernel_size=3, padding="same",stride=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(7*7*512, 4096),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 256),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.fc3 = nn.Sequential(
            nn.Linear(256, 1)
            )
       
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0),256*6*6)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out
        
model = CNN()
#CUDA not available
model.cuda()


In [ ]:
import torch.utils.data as utils


tensor_x = torch.Tensor(X) # transform to torch tensors
tensor_y = torch.Tensor(y)

my_dataset = utils.TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = utils.DataLoader(my_dataset) # create your dataloader

In [ ]:
X= np.load('./benign_X.npy')
for ix in range(200):
    plt.figure(ix/10)
    plt.subplot(1,10,(ix%10)+1)
    plt.imshow(X[ix],cmap='gray')
    plt.axis('off')